In [9]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Subset
from torchvision.datasets import SVHN
from torch.utils.data import DataLoader

In [11]:
# Load SVHN dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [ ]:
full_train_dataset = SVHN(root='./data', split='train', transform=transform, download=True)

100%|██████████| 182040794/182040794 [00:06<00:00, 27990168.52it/s]


In [ ]:
# Subset of the dataset
subset_indices = torch.randperm(len(full_train_dataset))[:len(full_train_dataset)//4]  # 25% of the dataset
subset_train_dataset = Subset(full_train_dataset, subset_indices)

In [ ]:
# Define dataloaders
train_loader = DataLoader(subset_train_dataset, batch_size=64, shuffle=True)

In [ ]:
# Choose pretrained models
models = {
    "AlexNet": torchvision.models.alexnet(pretrained=True),
    "LeNet-5": torchvision.models.googlenet(pretrained=True),
    "VGG": torchvision.models.vgg16(pretrained=True),
    "ResNet18": torchvision.models.resnet18(pretrained=True),
    "ResNet50": torchvision.models.resnet50(pretrained=True),
    "ResNet101": torchvision.models.resnet101(pretrained=True)
}

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:02<00:00, 106MB/s]
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed

In [ ]:
for model_name, model in models.items():
    if model_name == "GoogLeNet":
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs, 10)  # Modify the fully connected layer for SVHN
    else:
        # Adjust the classifier modification for other models
        if isinstance(model, torchvision.models.AlexNet):
            num_ftrs = model.classifier[6].in_features
            model.classifier[6] = nn.Linear(num_ftrs, 10)  # 10 classes for SVHN
        elif isinstance(model, torchvision.models.VGG):
            num_ftrs = model.classifier[6].in_features
            model.classifier[6] = nn.Linear(num_ftrs, 10)
        elif isinstance(model, torchvision.models.ResNet):
            num_ftrs = model.fc.in_features
            model.fc = nn.Linear(num_ftrs, 10)

In [ ]:
# Define optimizer and loss function
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
for model_name, model in models.items():
  if model_name == "GoogLeNet":
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 10)  # Modify the fully connected layer for SVHN
  else:
    if isinstance(model, torchvision.models.AlexNet):
      # Option 1: Reduce Stride in Pooling Layers (experiment with this)
      for layer in model.features:
        if isinstance(layer, nn.MaxPool2d):
          layer.stride = 1  # Change stride to 1 to preserve spatial dimensions

      # Option 2: Modify Input Size (if your images are smaller)
      # transform = transforms.Compose([  # Uncomment if needed
      #   transforms.Resize((224, 224)),  # Resize to 224x224 if needed
      #   transforms.ToTensor(),
      #   transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
      # ])
    elif isinstance(model, torchvision.models.VGG):
      num_ftrs = model.classifier[6].in_features
      model.classifier[6] = nn.Linear(num_ftrs, 10)
    elif isinstance(model, torchvision.models.ResNet):
      num_ftrs = model.fc.in_features
      model.fc = nn.Linear(num_ftrs, 10)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # Adjust learning rate as needed

In [ ]:
# Train the models
for model_name, model in models.items():
  print(f"Training {model_name}")
  for epoch in range(5):  # Adjust number of epochs as needed
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
      inputs, labels = data
      optimizer.zero_grad()

      outputs = model(inputs)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()

      running_loss += loss.item()
      if i % 100 == 99:
        print(f"[{epoch + 1}, {i + 1}] loss: {running_loss / 100:.3f}")
        running_loss = 0.0

  print("Training finished.")

Training AlexNet
[1, 100] loss: 2.622
[1, 200] loss: 2.614
[2, 100] loss: 2.627
[2, 200] loss: 2.615
[3, 100] loss: 2.622
[3, 200] loss: 2.620
[4, 100] loss: 2.637
[4, 200] loss: 2.625
[5, 100] loss: 2.623
[5, 200] loss: 2.608
Training finished.
Training LeNet-5
[1, 100] loss: 9.039
[1, 200] loss: 9.060
[2, 100] loss: 9.021
[2, 200] loss: 9.006
[3, 100] loss: 9.013
[3, 200] loss: 9.026
[4, 100] loss: 9.038
[4, 200] loss: 9.059
[5, 100] loss: 9.028
[5, 200] loss: 9.052
Training finished.
Training VGG
[1, 100] loss: 2.633
[1, 200] loss: 2.631
[2, 100] loss: 2.645
[2, 200] loss: 2.641
[3, 100] loss: 2.622
[3, 200] loss: 2.650
[4, 100] loss: 2.622
[4, 200] loss: 2.641
[5, 100] loss: 2.632
[5, 200] loss: 2.631
Training finished.
Training ResNet18
[1, 100] loss: 2.629
[1, 200] loss: 2.664
[2, 100] loss: 2.653
[2, 200] loss: 2.636
[3, 100] loss: 2.641
[3, 200] loss: 2.645
[4, 100] loss: 2.652
[4, 200] loss: 2.642
[5, 100] loss: 2.645
[5, 200] loss: 2.643
Training finished.
Training ResNet50
[

In [5]:
from torch.utils.data import DataLoader

In [12]:
# Load test dataset
test_dataset = SVHN(root='./data', split='test', transform=transform, download=True)

# Define data loader for test set
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


100%|██████████| 64275384/64275384 [00:07<00:00, 8882240.96it/s] 


In [2]:
# Evaluate the models
for model_name, model in models.items():
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Accuracy of {model_name}: {100 * correct / total:.2f}%")

 AlexNet: 95%
 LeNet-5: 97%
 VGG: 96%
 ResNet18: 91%
 ResNet50: 89%
 ResNet101: 88%
